# data exploration

## loading the data

In [1]:
from clef.utils.data_loading import load_datasets
from clef.utils.data_loading import write_trec_format_output

train, dev = load_datasets(preprocess=True, add_author_info=True)

out_dir = 'data-out/exp-add-author-info'

import json
data_grouped_by_id_dev = {}
for entry in dev:
    data_grouped_by_id_dev[entry['id']] = entry
# print(json.dumps(data_grouped_by_id['AuRED_142'], indent=2))

import json
data_grouped_by_id_train = {}
for entry in train:
    data_grouped_by_id_train[entry['id']] = entry
# print(json.dumps(data_grouped_by_id['AuRED_142'], indent=2))

loaded 96 training json lines and 32 dev json lines.


In [2]:
def diagnose_retrieval(filename, data_grouped_by_id):
    with open(filename, 'r') as file:
        prev_rumor = ''
        ev_from_trec = []
        rumor_dict = None
        for line in file:
            try:
                rumor_id,q0,evidence_id,rank,score,label = line.split('\t')
                
                if rumor_dict and prev_rumor != rumor_id:
                    ev_not_found = [e[2] for e in rumor_dict['evidence'] if e[1] not in ev_from_trec]
                    if ev_not_found:
                        print('evidence not found:')
                        for text in ev_not_found:
                            print(text)
                    else:
                        print('all evidence found!')
                    ev_from_trec = []
                    print('\n')


                rumor_dict = data_grouped_by_id[rumor_id]
                if int(rank) == 1:
                    print(rumor_dict['label'], ':', rumor_dict['rumor'])

                ev_author, ev_id, ev_text = '', '', ''
                ev_ids = [e[1] for e in rumor_dict['evidence']]

                for ev in rumor_dict['timeline']:
                    if evidence_id == ev[1]:
                        ev_author, ev_id, ev_text = ev
                        ev_from_trec += [evidence_id]
                    
                print('(*)\t' if evidence_id in ev_ids else '\t',
                    rumor_id,'\t',
                    evidence_id,'\t',
                    rank,'\t',
                    f'{" " if float(score) > 0 else ""}{float(score):.4f}','\t',
                    ev_text,'\t',
                    f'{ev_author}/status/{ev_id}','\t',
                    )
                prev_rumor = rumor_id
            except KeyError:
                pass

## print datasets with retrieved evidence next to it (per method)

In [3]:
# match up the retrieval scores from the dataset submission sample with the text from the tweets
tag = 'LUCENE'
print(tag)
diagnose_retrieval(f'{out_dir}/{tag}-dev.trec.txt', data_grouped_by_id_dev)

LUCENE
REFUTES : Naturalization decree in preparation: Lebanese passports for sale !
(*)	 AuRED_142 	 1555986659279360001 	 1 	  4.1100 	 Account: Lebanese Presidency
Description: The Presidency of the Lebanese Republic
Text: The Information Office of the Presidency of the Republic denies a false news broadcast by the MTV station about Baabda Palace preparing a decree naturalizing 4 000 people and recalls that it had denied yesterday the false information published by the French magazine Liberation about the same fabricated news 	 https://twitter.com/LBpresidency/status/1555986659279360001 	
(*)	 AuRED_142 	 1555424541509386240 	 2 	  1.5088 	 Account: Lebanese Presidency
Description: The Presidency of the Lebanese Republic
Text: The Information Office of the Presidency of the Republic: What was published by the French newspaper Liberation about the selling of Lebanese passports to non-Lebanese is false and baseless news 	 https://twitter.com/LBpresidency/status/1555424541509386240 	
	

In [4]:
# match up the retrieval scores from the dataset submission sample with the text from the tweets
tag = 'OPENAI'
print(tag)
diagnose_retrieval(f'{out_dir}/{tag}-dev.trec.txt', data_grouped_by_id_dev)

OPENAI
REFUTES : Naturalization decree in preparation: Lebanese passports for sale !
(*)	 AuRED_142 	 1555424541509386240 	 1 	  0.5714 	 Account: Lebanese Presidency
Description: The Presidency of the Lebanese Republic
Text: The Information Office of the Presidency of the Republic: What was published by the French newspaper Liberation about the selling of Lebanese passports to non-Lebanese is false and baseless news 	 https://twitter.com/LBpresidency/status/1555424541509386240 	
(*)	 AuRED_142 	 1555986659279360001 	 2 	  0.4839 	 Account: Lebanese Presidency
Description: The Presidency of the Lebanese Republic
Text: The Information Office of the Presidency of the Republic denies a false news broadcast by the MTV station about Baabda Palace preparing a decree naturalizing 4 000 people and recalls that it had denied yesterday the false information published by the French magazine Liberation about the same fabricated news 	 https://twitter.com/LBpresidency/status/1555986659279360001 	
	

In [5]:
tag = 'TFIDF'
print(tag)
diagnose_retrieval(f'{out_dir}/{tag}-dev.trec.txt', data_grouped_by_id_dev)

TFIDF
(*)	 AuRED_142 	 1555424541509386240 	 0 	  0.0985 	 Account: Lebanese Presidency
Description: The Presidency of the Lebanese Republic
Text: The Information Office of the Presidency of the Republic: What was published by the French newspaper Liberation about the selling of Lebanese passports to non-Lebanese is false and baseless news 	 https://twitter.com/LBpresidency/status/1555424541509386240 	
REFUTES : Naturalization decree in preparation: Lebanese passports for sale !
	 AuRED_142 	 1555126458191855616 	 1 	  0.0925 	 Account: Lebanese Presidency
Description: The Presidency of the Lebanese Republic
Text: President Aoun received the Ambassador of Finland Tarja Fernandes on a farewell visit on the occasion of the end of her diplomatic duties in Lebanon He thanked her for the efforts she made to strengthen and develop Lebanese-Finnish relations and wished her success in her new duties 	 https://twitter.com/LBpresidency/status/1555126458191855616 	
	 AuRED_142 	 15554954270044200

In [6]:
# match up the retrieval scores from the dataset submission sample with the text from the tweets

from utils.data_loading import clef_base_path
kgat_retrieve_sample = clef_base_path + '/submission_samples/KGAT_zeroShot_evidence_English_dev.txt'
print('KGAT BASELINE')
diagnose_retrieval(kgat_retrieve_sample, data_grouped_by_id_dev)

KGAT BASELINE
REFUTES : Naturalization decree in preparation: Lebanese passports for sale !
(*)	 AuRED_142 	 1555986659279360001 	 1 	 -0.7252 	 Account: Lebanese Presidency
Description: The Presidency of the Lebanese Republic
Text: The Information Office of the Presidency of the Republic denies a false news broadcast by the MTV station about Baabda Palace preparing a decree naturalizing 4 000 people and recalls that it had denied yesterday the false information published by the French magazine Liberation about the same fabricated news 	 https://twitter.com/LBpresidency/status/1555986659279360001 	
(*)	 AuRED_142 	 1555424541509386240 	 2 	 -0.8598 	 Account: Lebanese Presidency
Description: The Presidency of the Lebanese Republic
Text: The Information Office of the Presidency of the Republic: What was published by the French newspaper Liberation about the selling of Lebanese passports to non-Lebanese is false and baseless news 	 https://twitter.com/LBpresidency/status/1555424541509386